In [1]:
from glotaran_tools.specification_parser import parse_yml
from glotaran_core.model import Dataset
from glotaran_models.kinetic import KineticSeparableModel
import numpy as np
from lmfit import Parameters
# Imports for plotting

import matplotlib.pyplot as plt
%matplotlib notebook

test_parameter = [201e-5, 535e-6, 0.1, 10]
test_amps = [1,2,3]
times = np.asarray(np.arange(-50, 1500, 1.5))

fitspec = '''
type: kinetic

parameter: {}

compartments: [s1, s2]

megacomplexes:
    - label: mc1
      k_matrices: [k1]
      
k_matrices:
  - label: "k1"
    matrix: {{
      '("s1","s1")': 1,
      '("s2","s2")': 2,
}}

irf:
  - label: irf1
    type: gaussian
    center: 3
    width: 4
datasets: 
  - label: dataset1
    type: spectral
    megacomplexes: [mc1]
    path: 'data/dataset1.ascii'
    irf: irf1

'''
model = parse_yml(fitspec.format(test_parameter))
print(model)

Modeltype: Kinetic

Parameter
---------

Index: 1 Initial Value: 0.00201 Label: None
Index: 2 Initial Value: 0.000535 Label: None
Index: 3 Initial Value: 0.1 Label: None
Index: 4 Initial Value: 10.0 Label: None

Megacomplexes
-------------

Label: mc1
K-Matrices: ['k1']

Datasets
--------

Dataset 'dataset1'

	Dataset Scaling: None
	Initial Concentration: None
	Megacomplexes: ['mc1']

	Irf: irf1


K-Matrices
----------

Label: k1
Matrix:
[[2 0]
 [0 1]]


IRFs
----

Label: irf1 Type: 'Gaussian' Center: [3] Width: [4] Center Dispersion: [] Width Dispersion [] Scale: [], Nomalize: True



In [2]:
simparams = Parameters()
simparams.add("p1", 101e-5)
simparams.add("p2",606e-6)
simparams.add("p3",0.3)
simparams.add("p4",20)
simparams.pretty_print()

Name     Value      Min      Max   Stderr     Vary     Expr
p1   0.00101     -inf      inf     None     True     None
p2  0.000606     -inf      inf     None     True     None
p3       0.3     -inf      inf     None     True     None
p4        20     -inf      inf     None     True     None


In [3]:
fitmodel = KineticSeparableModel(model)
fitmodel.get_initial_fitting_parameter().pretty_print()

Name     Value      Min      Max   Stderr     Vary     Expr
p1   0.00201     -inf      inf     None     True     None
p2  0.000535     -inf      inf     None     True     None
p3       0.1     -inf      inf     None     True     None
p4        10     -inf      inf     None     True     None


/usr/lib64/python3.5/site-packages/pyopencl-2016.2-py3.5-linux-x86_64.egg/pyopencl/cffi_cl.py:1469: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


In [4]:
data = fitmodel.eval(simparams, *times, **{'dataset':'dataset1'})
noisy_data = fitmodel.eval(simparams, *times, **{'dataset':'dataset1', 'noise':True, 'noise_std_dev':0.01})
fig = plt.figure()
plt.plot(times, data[:, 0])
plt.plot(times, noisy_data[:, 0])

KeyError: 'dataset1'

In [ ]:
result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), 
                      *times, 
                      **{"data": data, 'verbose':1}
                     )
result.best_fit_parameter.pretty_print()
fitted_data = fitmodel.eval(result.best_fit_parameter, *times, **{'dataset':'dataset1'})
reconstructed_data = result.eval(*times, **{"data": data})
print('Fitted Amplitudes: {}'.format(result.e_matrix(*times, **{"data": data})))
fig = plt.figure()
plt.plot(times, data[:, 0])
plt.plot(times, fitted_data[:, 0])
plt.plot(times, reconstructed_data[:, 0])

In [ ]:
noise_result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), 
                      *times, 
                      **{"data": noisy_data, 'verbose':1}
                     )
noise_result.best_fit_parameter.pretty_print()
print('Fitted Amplitudes: {}'.format(noise_result.e_matrix(*times, **{"data": data})))
fitted_data = fitmodel.eval(noise_result.best_fit_parameter, *times, **{'dataset':'dataset1'})
reconstructed_data = noise_result.eval(*times, **{"data": data})

fig = plt.figure()
plt.plot(times, noisy_data[:, 0])
plt.plot(times, fitted_data[:, 0])
plt.plot(times, reconstructed_data[:, 0])